* reference : https://www.kaggle.com/nafisur/predictive-maintenance-using-lstm-on-sensor-data
* data : https://www.kaggle.com/c/predictive-maintenance1/data

In [32]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import warnings
warnings.filterwarnings('ignore')
import keras
import keras.backend as K
from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/test_label_sample.csv
/kaggle/input/feature.xlsx
/kaggle/input/train_label.csv


In [33]:
#load files
train = pd.read_csv("/kaggle/input/train_label.csv")
features = pd.read_excel("/kaggle/input/feature.xlsx")
test = pd.read_csv("/kaggle/input/test_label_sample.csv")

In [14]:
train

,date,label
0,3/5/2015,NaN
1,4/5/2015,0.0
2,5/5/2015,0.0
3,6/5/2015,0.0
4,7/5/2015,0.0
...,...,...
679,20/3/2017,0.0
680,21/3/2017,0.0
681,22/3/2017,0.0
682,23/3/2017,1.0


In [34]:
train['date']= pd.to_datetime(train['date'],format='%d/%m/%Y') 
train.shape

(684, 2)

In [16]:
train

,date,label
0,2015-05-03,NaN
1,2015-05-04,0.0
2,2015-05-05,0.0
3,2015-05-06,0.0
4,2015-05-07,0.0
...,...,...
679,2017-03-20,0.0
680,2017-03-21,0.0
681,2017-03-22,0.0
682,2017-03-23,1.0


In [35]:
test['date']= pd.to_datetime(test['date'],format='%d/%m/%Y') 
test.shape

(300, 2)

In [23]:
features

,Unnamed: 0,Unnamed: 1,count,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131
0,NaN,date,136088194,136088202,136088394,136088802,136089546,136110468,136216674,136222202,...,1.362250e+08,136675426.0,136675626.0,1.366766e+08,136676650.0,136676666.0,136676682.0,136676698.0,136676714.0,136676754.0
1,0.0,2015-05-03 00:00:00,0,0,0,0,1,504,0,2,...,9.056261e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2015-05-04 00:00:00,0,0,0,0,0,2250,0,1,...,2.540209e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,2015-05-05 00:00:00,0,0,0,0,0,1700,0,2,...,1.360972e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,2015-05-06 00:00:00,0,0,0,0,0,2261,0,10,...,2.612264e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,978.0,2018-01-13 00:00:00,38,0,0,13,0,15,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
980,979.0,2018-01-14 00:00:00,14,0,29,4,0,15,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
981,980.0,2018-01-15 00:00:00,5,0,0,0,0,1,1,2,...,1.757063e+04,NaN,NaN,1.244734e+04,NaN,NaN,NaN,NaN,NaN,NaN
982,981.0,2018-01-16 00:00:00,23,0,0,0,1,19,0,3,...,2.681461e+04,NaN,NaN,2.240069e+04,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
new_header = features.iloc[0] #grab the first row for the header
features = features[1:] #take the data less the header row
features.columns = new_header #set the header row as the df header
features['date'] = features['date'].dt.date

In [26]:
features

,NaN,date,136088194,136088202,136088394,136088802,136089546,136110468,136216674,136222202,...,136225010.0,136675426.0,136675626.0,136676578.0,136676650.0,136676666.0,136676682.0,136676698.0,136676714.0,136676754.0
1,0.0,2015-05-03,0,0,0,0,1,504,0,2,...,9056.261199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2015-05-04,0,0,0,0,0,2250,0,1,...,25402.087770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,2015-05-05,0,0,0,0,0,1700,0,2,...,13609.724304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,2015-05-06,0,0,0,0,0,2261,0,10,...,26122.640408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,2015-05-07,0,0,0,0,0,2950,0,5,...,25499.201836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,978.0,2018-01-13,38,0,0,13,0,15,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
980,979.0,2018-01-14,14,0,29,4,0,15,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
981,980.0,2018-01-15,5,0,0,0,0,1,1,2,...,17570.630928,NaN,NaN,12447.336848,NaN,NaN,NaN,NaN,NaN,NaN
982,981.0,2018-01-16,23,0,0,0,1,19,0,3,...,26814.610131,NaN,NaN,22400.688963,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
error_id_count=features.iloc[:,1:28]
error_id_count['date']= pd.to_datetime(error_id_count['date'],format='%Y-%m-%d')
error_id_count.shape

(983, 27)

In [37]:
df_train= pd.merge(error_id_count, train, on='date')
df_train.columns = ['date', 'e1', 'e2', 'e3','e4', 'e5', 'e6', 'e7', 'e8', 'e9', 'e10', 'e11', 'e12', 'e13', 'e14',
                     'e15', 'e16', 'e17', 'e18', 'e19', 'e20', 'e21','e22','e23','e24','e25','e26','label']
df_train.head()

,date,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e18,e19,e20,e21,e22,e23,e24,e25,e26,label
0,2015-05-03,0,0,0,0,1,504,0,2,8,...,0,0,0,0,0,0,0,0,0,NaN
1,2015-05-04,0,0,0,0,0,2250,0,1,2,...,0,0,0,0,0,0,0,0,0,0.0
2,2015-05-05,0,0,0,0,0,1700,0,2,5,...,0,0,0,0,0,0,0,0,0,0.0
3,2015-05-06,0,0,0,0,0,2261,0,10,7,...,0,0,0,0,0,0,0,0,0,0.0
4,2015-05-07,0,0,0,0,0,2950,0,5,8,...,0,0,0,0,0,0,0,0,0,0.0


In [38]:
df_test = pd.merge(error_id_count, test, on='date')
df_test.columns = ['date', 'e1', 'e2', 'e3','e4', 'e5', 'e6', 'e7', 'e8', 'e9', 'e10', 'e11', 'e12', 'e13', 'e14',
                     'e15', 'e16', 'e17', 'e18', 'e19', 'e20', 'e21','e22','e23','e24','e25','e26','label']
df_test.head()

,date,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e18,e19,e20,e21,e22,e23,e24,e25,e26,label
0,2017-03-25,1,0,0,0,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0.130578
1,2017-03-26,0,0,0,0,0,3,0,1,0,...,0,0,0,0,0,0,0,0,0,0.124545
2,2017-03-27,0,0,0,0,0,3,0,2,0,...,0,0,1,0,0,0,0,0,0,0.111402
3,2017-03-28,2,0,0,0,0,12,0,3,10,...,0,0,0,0,0,0,0,0,0,0.115621
4,2017-03-29,0,0,0,0,1,60,0,30,25,...,0,0,1,0,0,0,0,0,0,0.071451


In [39]:
features_col_name=[ 'e1', 'e2', 'e3','e4', 'e5', 'e6', 'e7', 'e8', 'e9', 'e10', 'e11', 'e12', 'e13', 'e14',
                     'e15', 'e16', 'e17', 'e18', 'e19', 'e20', 'e21','e22','e23','e24','e25','e26']
target_col_name='label'

In [40]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()
# We use min max scaler to have the data between   0 and 1
df_train[features_col_name]=sc.fit_transform(df_train[features_col_name])
# For normalization, this means the training data will be used to estimate the minimum and maximum observable values. This is done by calling the fit() function.
df_test[features_col_name]=sc.transform(df_test[features_col_name])
# This means you can use the normalized data to train your model.
# This is done by calling the transform() function.

In [45]:
def gen_sequence(id_df, seq_length, seq_cols):
    df_zeros=pd.DataFrame(np.zeros((seq_length-1,id_df.shape[1])),columns=id_df.columns)
    id_df=df_zeros.append(id_df,ignore_index=True)
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    lstm_array=[]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        lstm_array.append(data_array[start:stop, :])
    return np.array(lstm_array)

In [46]:
def gen_label(id_df, seq_length, seq_cols,label):
    df_zeros=pd.DataFrame(np.zeros((seq_length-1,id_df.shape[1])),columns=id_df.columns)
    id_df=df_zeros.append(id_df,ignore_index=True)
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    y_label=[]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        y_label.append(id_df[label][stop])
    return np.array(y_label)

In [48]:
# timestamp or window size
seq_length=50
seq_cols=features_col_name
seq_cols

['e1',
 'e2',
 'e3',
 'e4',
 'e5',
 'e6',
 'e7',
 'e8',
 'e9',
 'e10',
 'e11',
 'e12',
 'e13',
 'e14',
 'e15',
 'e16',
 'e17',
 'e18',
 'e19',
 'e20',
 'e21',
 'e22',
 'e23',
 'e24',
 'e25',
 'e26']

In [53]:
X_train=gen_sequence(df_train, seq_length, seq_cols)
print(X_train.shape)
# generate y_train
y_train=gen_label(df_train, 50, seq_cols,'label')
print(y_train.shape)
print(y_train)

(683, 50, 26)
(683,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [54]:
X_test=gen_sequence(df_test, seq_length, seq_cols)
print(X_test.shape)
# generate y_train
y_test=gen_label(df_test, 50, seq_cols,'label')
print(y_test.shape)

(298, 50, 26)
(298,)


**LSTM NETWORK**

In [56]:
nb_features =X_train.shape[2]
timestamp=seq_length

model = Sequential()

model.add(LSTM(
         input_shape=(timestamp, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 100)           50800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 81,051
Trainable params: 81,051
Non-trainable params: 0
_________________________________________________________________


In [68]:
model.fit(X_train, y_train, epochs=10, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = [EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])

Train on 648 samples, validate on 35 samples
Epoch 1/10
648/648 [==============================] - 0s 634us/step - loss: 0.2275 - accuracy: 0.9414 - val_loss: 0.5555 - val_accuracy: 0.8286
Epoch 2/10
648/648 [==============================] - 0s 625us/step - loss: 0.2277 - accuracy: 0.9414 - val_loss: 0.5627 - val_accuracy: 0.8286


In [69]:
# training metrics
scores = model.evaluate(X_train, y_train, verbose=1, batch_size=200)
print('Accurracy: {}'.format(scores[1]))

683/683 [==============================] - 0s 222us/step
Accurracy: 0.9355783462524414


In [62]:
y_pred=model.predict_proba(X_test)
y_pred

array([[0.02749231],
       [0.02747294],
       [0.0274123 ],
       [0.0272961 ],
       [0.02674985],
       [0.02601838],
       [0.02530059],
       [0.02467018],
       [0.02400699],
       [0.02345836],
       [0.0228973 ],
       [0.02243638],
       [0.02201903],
       [0.0216192 ],
       [0.02123567],
       [0.02090302],
       [0.02060291],
       [0.02032799],
       [0.02005777],
       [0.0197916 ],
       [0.01955575],
       [0.0192993 ],
       [0.01902103],
       [0.018758  ],
       [0.01852795],
       [0.01828316],
       [0.01805687],
       [0.01784679],
       [0.01763508],
       [0.01743683],
       [0.01725319],
       [0.01707539],
       [0.01688904],
       [0.01671836],
       [0.01655993],
       [0.01640883],
       [0.01626679],
       [0.01610371],
       [0.01594782],
       [0.01580191],
       [0.01566094],
       [0.0154928 ],
       [0.01537037],
       [0.01524073],
       [0.01511356],
       [0.01500055],
       [0.01486012],
       [0.014